In [0]:
%pip install databricks-cli
%pip install databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
df_eff_dates = spark.sql("""
    SELECT DISTINCT to_date(int_effective_date) AS int_effective_date
    FROM gap_catalog.int_owner.int_sbd_clients
    ORDER BY to_date(int_effective_date)
""")

increments = {str(row.int_effective_date): f"B{str(row.int_effective_date).replace('-', '')}" for row in df_eff_dates.collect()}

In [0]:
from databricks_cli.jobs.api import JobsApi
from databricks_cli.runs.api import RunsApi
from databricks_cli.sdk.api_client import ApiClient
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import JobSettings
import time

token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

workspace_client = WorkspaceClient()
job_id = "361542017502456"

new_warehouse_id = "63bb9c3e8676522d"
existing_job = workspace_client.jobs.get(job_id=job_id)
new_tasks = []
for task in existing_job.settings.tasks:
    if task.sql_task:
        task.sql_task.warehouse_id = new_warehouse_id
        new_tasks.append(task)

for date, partition in increments.items():
    # Update tags
    workspace_client.jobs.update(
        job_id=job_id,
        new_settings=JobSettings(
            tags={
                "effective_date": date,
                "etl": "LD_DWH_EGR",
                "cluster_size": "XS"
            }
        )
    )
    
    # Trigger the job
    try:
        run = workspace_client.jobs.run_now_and_wait(
                    job_id=job_id,
                    job_parameters={
                        "p_effective_date": date,
                        "p_process_key": "-1",
                        "p_load_partition": partition
                    }
                )
        print("Run finished with state:", run.state.life_cycle_state, run.state.result_state, run.execution_duration)
    except Exception as e:
        print(e)
    
    

In [0]:
workspace_client = WorkspaceClient()
job_id = "361542017502456"

new_warehouse_id = "94a4cb87c4f0d125"
existing_job = workspace_client.jobs.get(job_id=job_id)
new_tasks = []
for task in existing_job.settings.tasks:
    if task.sql_task:
        task.sql_task.warehouse_id = new_warehouse_id
        new_tasks.append(task)

# Update tags
workspace_client.jobs.update(
    job_id=job_id,
    new_settings=JobSettings(
        tasks=new_tasks,
        tags={
            "effective_date": date,
            "etl": "LD_DWH_EGR",
            "cluster_size": "S"
        }
    )
)

In [0]:
%sql
SELECT operationMetrics['numOutputRows'] AS affected_rows
FROM (DESCRIBE HISTORY gap_catalog.dwh_owner_tmp.parties) WHERE operation='MERGE' ORDER BY version DESC LIMIT 1

In [0]:
%sql
DESCRIBE HISTORY gap_catalog.dwh_owner_tmp.parties